In [65]:
import numpy as np
import matplotlib.pyplot as plt
import functools as ft
from scipy.linalg import expm
from scipy.stats import unitary_group

# Projected mixed state ensembles: Model 2

In [72]:
# Parameters
N_A = 1
N_B = 1
N = N_A + N_B

def random_state(d):
    '''Produces a Haar random state in d-dimensional Hilbert space as a numpy array.'''
    U = unitary_group.rvs(d)
    psi0 = np.zeros(d, dtype='complex128')
    psi0[0] = 1
    
    return U @ psi0

def Hamming(str_a, str_b):
    '''Returns the Hamming distance of two equal length strings'''
    Hamming = 0
    for i in range(len(str_a)):
        if str_a[i] != str_b[i]:
            Hamming += 1
    return Hamming

def bitstring(i, N_B):
    '''Returns the bitstring of N_B bits that corresponds to the base-10 integer i, for i between 0 and 2**N_B.'''
    return format(i, f'0{N_B}b')

def error_prob(m, m_prime, N_B, p):
    '''
    For single independent bitflip errors with probability p, returns the probability of reading m_prime for a true outcome m,
    for m and m_prime input as base-10 integers between 0 and 2**N_B.
    '''
    str_m = format(m, f'0{N_B}b')
    str_m_prime = format(m_prime, f'0{N_B}b')
    num_errors = Hamming(str_m, str_m_prime)
    prob = (p ** num_errors) * ((1-p) ** (N_B - num_errors))

    return prob

# From previous work: get projected ensemble on A from a state in AB

def Get_PrEns(Psi, N_A, N_B):
    PrEns = np.empty((2**N_B, 2)).tolist()

    for i in range(2**N_B):
        PsiA = Psi.reshape(2**N_A, 2**N_B)[:,i]
        PrEns[i][0] = np.linalg.norm(PsiA) ** 2
        PrEns[i][1] = PsiA / np.linalg.norm(PsiA)
    
    return PrEns

def Get_MixEns(PrEns, N_A, N_B, p):
    '''From a pure ensemble, return a mixed ensemble given an independent bitflip error p'''
    MixEns = np.empty((2 ** N_B, 2)).tolist()

    for m_prime in range(2 ** N_B):
        rho = np.zeros((2 ** N_A, 2 ** N_A), dtype='complex128')
        p_m_prime = 0

        for m in range(2 ** N_B):
            cond_prob = error_prob(m, m_prime, N_B, p)
            p_m_prime += cond_prob * PrEns[m][0]
            rho += cond_prob * np.outer(np.conjugate(PrEns[m][1]), PrEns[m][1])

        MixEns[m_prime][0] = p_m_prime
        MixEns[m_prime][1] = rho

    return MixEns

In [71]:
N_A = 1
N_B = 2
p = 0.1

Psi = random_state(2 ** (N_A + N_B))
Projected_ensemble = Get_PrEns(Psi, N_A, N_B)
Mixed_ensemble = Get_MixEns(Projected_ensemble, N_A, N_B, p)

print(Mixed_ensemble)

[[0.340613983760482, array([[ 0.84877607+0.j        , -0.15355461-0.06639505j],
       [-0.15355461+0.06639505j,  0.15122393+0.j        ]])], [0.10804238281824599, array([[ 0.34657808+0.j        , -0.20811278+0.26969462j],
       [-0.20811278-0.26969462j,  0.65342192+0.j        ]])], [0.3799084684808134, array([[ 0.47946224+0.j        , -0.41987235+0.09737188j],
       [-0.41987235-0.09737188j,  0.52053776+0.j        ]])], [0.17143516494045843, array([[ 0.58303317+0.j        , -0.3550078 -0.20790181j],
       [-0.3550078 +0.20790181j,  0.41696683+0.j        ]])]]
